In [59]:
import numpy as np
import pandas as pd
import random as rd

In [60]:
#CREAMOS LOS REGISTROS SINTETICOS

#diccionario con 104 registros de tarifa con 20 registros cada una de total de kWh consumido en el periodo de facturacion

np.random.seed = 1309

consumos = {}

for tarifa in range(104):
    consumos[tarifa] = []
    for registro in range(21):
        valores = np.random.randint(100, 300)
        consumos[tarifa].append(valores)

#diccionario con registros de tarifa con 104 registros cada una de pesos de consumo para la pvpc y tarifas por tramos


pesos_3t = {}
for tarifa in range(104):
    pesos_3t[tarifa] = []
    while len(pesos_3t[tarifa]) < 21:
        a,b = np.random.randint(0,101,2)
        c = 100 - a - b
        while c < 0:
            a,b = np.random.randint(0,101,2)
            c = 100 - a - b
        pesos_3t[tarifa].append([a,b,c])

#diccionario con registros de tarifa con 105 registros cada una de pesos de consumo para tarifas de discriminacion horaria


pesos_2t = {}
for tarifa in range(104):
    pesos_2t[tarifa] = []
    while len(pesos_2t[tarifa]) < 21:
        d = np.random.randint(0,101)
        e = 100 - d
        pesos_2t[tarifa].append([d,e])

In [61]:
#APLANAMOS LOS DICCIONARIOS

registros = []
for tarifa, valores in consumos.items():
    for valor in valores:
        registros.append({'tarifa': tarifa, 'consumo_kWh': valor})

tramos_3t = []
for tarifa, valores in pesos_3t.items():
    for valor in valores:
        tramos_3t.append({'tarifa': tarifa, 'pesos': valor})

tramos_2t = []
for tarifa, valores in pesos_2t.items():
    for valor in valores:
        tramos_2t.append({'tarifa': tarifa, 'pesos': valor})

In [62]:
totales = pd.DataFrame(registros)

In [63]:
totales['%consumo_valle'] = None
totales['%consumo_llano'] = None
totales['%consumo_punta'] = None
totales['%consumo_8h'] = None
totales['%consumo_16h'] = None

In [64]:
for index, i in enumerate(tramos_3t):
        totales.loc[index, '%consumo_valle'] = i['pesos'][0]
        totales.loc[index, '%consumo_llano'] = i['pesos'][1]
        totales.loc[index, '%consumo_punta'] = i['pesos'][2]

for index, i in enumerate(tramos_2t):
        totales.loc[index, '%consumo_8h'] = i['pesos'][0]
        totales.loc[index, '%consumo_16h'] = i['pesos'][1]

In [65]:
totales

,tarifa,consumo_kWh,%consumo_valle,%consumo_llano,%consumo_punta,%consumo_8h,%consumo_16h
0,0,234,32,31,37,35,65
1,0,258,50,15,35,65,35
2,0,271,9,25,66,63,37
3,0,203,82,13,5,50,50
4,0,215,62,6,32,79,21
...,...,...,...,...,...,...,...
2179,103,275,15,24,61,33,67
2180,103,140,40,37,23,91,9
2181,103,282,66,33,1,12,88
2182,103,260,72,28,0,72,28


In [66]:
totales = totales.rename(columns={'tarifa': 'id_tarifa'})

In [67]:
# Traigo la base de datos
df = pd.read_csv('data\database_desafio.csv')

In [68]:
df.head()

,id,compañia,tarifa,tp_punta_euros/kwh/dia,tp_valle_euros/kwh/dia,peaje_tran_dist_pot_valle,peaje_tran_dist_pot_punta,peaje_tran_dist_ener_valle,peaje_tran_dist_ener_llano,peaje_tran_dist_ener_punta,...,tc_llano_euros/kwh/hora,tc_punta_euros/kwh/hora,tc_8h_euros/kwh/hora,tc_16h_euros/kwh/hora,mercado,tipo,otros,alquiler_equipo_euros/dia,impuesto_actual,bono_social_euro/dia
0,0,Repsol,Mis 10 Horas,"0,0681918","0,0681918","0,000527","0,0081915","0,002195","0,008779","0,043893",...,NaN,NaN,"0,086415","0,172830",Libre,DH,NaN,"0,026557","5,11",NaN
1,1,Iberdrola,Plan Elige 8 Horas,"0,1081918","0,0424110","0,000527","0,0081915","0,002195","0,008779","0,043893",...,NaN,NaN,"0,096700","0,241000",Libre,DH,NaN,"0,026557","5,11",NaN
2,2,Iberdrola,Plan Noche,"0,1081918","0,0424110","0,000527","0,0081915","0,002195","0,008779","0,043893",...,NaN,NaN,"0,109000","0,239000",Libre,DH,NaN,"0,026557","5,11",NaN
3,3,Iberdrola,Plan Familias,"0,1081918","0,0424110","0,000527","0,0081915","0,002195","0,008779","0,043893",...,NaN,NaN,"0,103000","0,189000",Libre,DH,NaN,"0,026557","5,11",NaN
4,4,Iberdrola,Plan Verano,"0,1081918","0,0424110","0,000527","0,0081915","0,002195","0,008779","0,043893",...,NaN,NaN,"0,099000","0,219000",Libre,DH,NaN,"0,026557","5,11",NaN


In [69]:
# Uno los datos sinteticos con las tarifas
clientes_tarifas = pd.merge(totales, df, left_on='id_tarifa', right_on='id', how='left')

In [75]:
# Para que se vean todas las columnas
pd.set_option('display.max_columns', None)

In [76]:
clientes_tarifas

,id_tarifa,consumo_kWh,%consumo_valle,%consumo_llano,%consumo_punta,%consumo_8h,%consumo_16h,id,compañia,tarifa,tp_punta_euros/kwh/dia,tp_valle_euros/kwh/dia,peaje_tran_dist_pot_valle,peaje_tran_dist_pot_punta,peaje_tran_dist_ener_valle,peaje_tran_dist_ener_llano,peaje_tran_dist_ener_punta,termino_consumo_euros/kwh/hora,tc_valle_euros/kwh/hora,tc_llano_euros/kwh/hora,tc_punta_euros/kwh/hora,tc_8h_euros/kwh/hora,tc_16h_euros/kwh/hora,mercado,tipo,otros,alquiler_equipo_euros/dia,impuesto_actual,bono_social_euro/dia
0,0,234,32,31,37,35,65,0,Repsol,Mis 10 Horas,"0,0681918","0,0681918","0,000527","0,0081915","0,002195","0,008779","0,043893",NaN,NaN,NaN,NaN,"0,086415","0,172830",Libre,DH,NaN,"0,026557","5,11",NaN
1,0,258,50,15,35,65,35,0,Repsol,Mis 10 Horas,"0,0681918","0,0681918","0,000527","0,0081915","0,002195","0,008779","0,043893",NaN,NaN,NaN,NaN,"0,086415","0,172830",Libre,DH,NaN,"0,026557","5,11",NaN
2,0,271,9,25,66,63,37,0,Repsol,Mis 10 Horas,"0,0681918","0,0681918","0,000527","0,0081915","0,002195","0,008779","0,043893",NaN,NaN,NaN,NaN,"0,086415","0,172830",Libre,DH,NaN,"0,026557","5,11",NaN
3,0,203,82,13,5,50,50,0,Repsol,Mis 10 Horas,"0,0681918","0,0681918","0,000527","0,0081915","0,002195","0,008779","0,043893",NaN,NaN,NaN,NaN,"0,086415","0,172830",Libre,DH,NaN,"0,026557","5,11",NaN
4,0,215,62,6,32,79,21,0,Repsol,Mis 10 Horas,"0,0681918","0,0681918","0,000527","0,0081915","0,002195","0,008779","0,043893",NaN,NaN,NaN,NaN,"0,086415","0,172830",Libre,DH,NaN,"0,026557","5,11",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2179,103,275,15,24,61,33,67,103,Aduriz Energia,coste + fija,"0,0800388","0,0031407","0,000527","0,0081915","0,002195","0,008779","0,043893",NaN,"0,093282","0,129895","0,179284",NaN,NaN,Libre,TRAMOS,"3,72","0,026557","5,11",NaN
2180,103,140,40,37,23,91,9,103,Aduriz Energia,coste + fija,"0,0800388","0,0031407","0,000527","0,0081915","0,002195","0,008779","0,043893",NaN,"0,093282","0,129895","0,179284",NaN,NaN,Libre,TRAMOS,"3,72","0,026557","5,11",NaN
2181,103,282,66,33,1,12,88,103,Aduriz Energia,coste + fija,"0,0800388","0,0031407","0,000527","0,0081915","0,002195","0,008779","0,043893",NaN,"0,093282","0,129895","0,179284",NaN,NaN,Libre,TRAMOS,"3,72","0,026557","5,11",NaN
2182,103,260,72,28,0,72,28,103,Aduriz Energia,coste + fija,"0,0800388","0,0031407","0,000527","0,0081915","0,002195","0,008779","0,043893",NaN,"0,093282","0,129895","0,179284",NaN,NaN,Libre,TRAMOS,"3,72","0,026557","5,11",NaN


In [78]:
# comprobamos por tarifas si el merge es como debe

clientes_tarifas[clientes_tarifas['id_tarifa'] == 12]

,id_tarifa,consumo_kWh,%consumo_valle,%consumo_llano,%consumo_punta,%consumo_8h,%consumo_16h,id,compañia,tarifa,tp_punta_euros/kwh/dia,tp_valle_euros/kwh/dia,peaje_tran_dist_pot_valle,peaje_tran_dist_pot_punta,peaje_tran_dist_ener_valle,peaje_tran_dist_ener_llano,peaje_tran_dist_ener_punta,termino_consumo_euros/kwh/hora,tc_valle_euros/kwh/hora,tc_llano_euros/kwh/hora,tc_punta_euros/kwh/hora,tc_8h_euros/kwh/hora,tc_16h_euros/kwh/hora,mercado,tipo,otros,alquiler_equipo_euros/dia,impuesto_actual,bono_social_euro/dia
252,12,145,19,55,26,23,77,12,Endesa,ConectaEndesa,"0,0997032","0,0327914","0,000527","0,0081915","0,002195","0,008779","0,043893","0,095700",NaN,NaN,NaN,NaN,NaN,Libre,FIJO,NaN,"0,026557","5,11",NaN
253,12,204,11,7,82,80,20,12,Endesa,ConectaEndesa,"0,0997032","0,0327914","0,000527","0,0081915","0,002195","0,008779","0,043893","0,095700",NaN,NaN,NaN,NaN,NaN,Libre,FIJO,NaN,"0,026557","5,11",NaN
254,12,181,9,44,47,2,98,12,Endesa,ConectaEndesa,"0,0997032","0,0327914","0,000527","0,0081915","0,002195","0,008779","0,043893","0,095700",NaN,NaN,NaN,NaN,NaN,Libre,FIJO,NaN,"0,026557","5,11",NaN
255,12,280,7,73,20,76,24,12,Endesa,ConectaEndesa,"0,0997032","0,0327914","0,000527","0,0081915","0,002195","0,008779","0,043893","0,095700",NaN,NaN,NaN,NaN,NaN,Libre,FIJO,NaN,"0,026557","5,11",NaN
256,12,116,7,14,79,69,31,12,Endesa,ConectaEndesa,"0,0997032","0,0327914","0,000527","0,0081915","0,002195","0,008779","0,043893","0,095700",NaN,NaN,NaN,NaN,NaN,Libre,FIJO,NaN,"0,026557","5,11",NaN
257,12,276,30,43,27,17,83,12,Endesa,ConectaEndesa,"0,0997032","0,0327914","0,000527","0,0081915","0,002195","0,008779","0,043893","0,095700",NaN,NaN,NaN,NaN,NaN,Libre,FIJO,NaN,"0,026557","5,11",NaN
258,12,274,65,5,30,31,69,12,Endesa,ConectaEndesa,"0,0997032","0,0327914","0,000527","0,0081915","0,002195","0,008779","0,043893","0,095700",NaN,NaN,NaN,NaN,NaN,Libre,FIJO,NaN,"0,026557","5,11",NaN
259,12,118,27,55,18,3,97,12,Endesa,ConectaEndesa,"0,0997032","0,0327914","0,000527","0,0081915","0,002195","0,008779","0,043893","0,095700",NaN,NaN,NaN,NaN,NaN,Libre,FIJO,NaN,"0,026557","5,11",NaN
260,12,226,13,18,69,97,3,12,Endesa,ConectaEndesa,"0,0997032","0,0327914","0,000527","0,0081915","0,002195","0,008779","0,043893","0,095700",NaN,NaN,NaN,NaN,NaN,Libre,FIJO,NaN,"0,026557","5,11",NaN
261,12,219,26,13,61,7,93,12,Endesa,ConectaEndesa,"0,0997032","0,0327914","0,000527","0,0081915","0,002195","0,008779","0,043893","0,095700",NaN,NaN,NaN,NaN,NaN,Libre,FIJO,NaN,"0,026557","5,11",NaN
